masih ada duplikat di bulan 11/12

In [1]:
import utils
import pandas as pd
import calendar
import re

In [2]:
pd.set_option('display.max_columns', None)

In [3]:
# 1
# open files
arr = utils.fileList("rsc/")
arr.sort()
lists = list()
unused_columns = ['nilai_kerugian',
'jumlah_tempat_pengungsian',
'jumlah_pengungsi_tertinggi',
'jumlah_luka_ringan',
'jumlah_luka_berat',
'jumlah_hilang',
'jumlah_meninggal',
'lama_genangan',
'ketinggian_air',
'jumlah_terdampak_jiwa',
'jumlah_terdampak_kk',
'jumlah_terdampak_rw',
'jumlah_terdampak_rt',
'rw']
for item in arr:
    csv = utils.readCsv("rsc/" + item)
    for unused in unused_columns:
        csv.drop(unused, axis=1, inplace=True)
    csv.drop_duplicates(subset=None, keep='first', inplace=False)
    lists.append(csv)


In [4]:
# 2
print(lists[0].columns)

Index(['kota_administrasi', 'kecamatan', 'kelurahan', 'tanggal_kejadian'], dtype='object')


In [5]:
# 3
nama_benahan = ["PULOGADUNG", "CILIINCING", "MAMPANG"]
for bulan in lists:
    for i in range(len(nama_benahan)):
        if i == 0:
            bulan.loc[bulan['kecamatan'] == nama_benahan[i], 'kecamatan'] = "PULO GADUNG"
        elif i == 1:
            bulan.loc[bulan['kecamatan'] == nama_benahan[i], 'kecamatan'] = "CILINCING"
        elif i == 2:
            bulan.loc[bulan['kecamatan'] == nama_benahan[i], 'kecamatan'] = "MAMPANG PRAPATAN"

In [6]:
# 4
loc = [10,11]
for l in loc:
    rows = lists[l]['tanggal_kejadian']
    # print(type(rows), l)
    for row in rows:
        # print(type(row))
        # print(row)
        if "-" in row:
            # if isinstance(row, str):
            #     dd = row[-2:]
            # else:
            #     dd = row[-2:].copy()

            dd = str(row[-2:])
            # print(dd, type(dd))
            lists[l].loc[lists[l]['tanggal_kejadian'] == row, 'tanggal_kejadian'] = dd
        

In [7]:
# 5
# normalize table in tanggal_kejadian column
data_normaliasi_tanggal = list()
# re.search(r'[0-9]{2}, txt')

def ubah_format(kata : str) -> str:
    if len(kata) == 1:
        return "0"+kata
    else:
        return kata



for bulan in lists:
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].str.split(',')
    bulan = bulan.explode('tanggal_kejadian').reset_index(drop=True)
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].apply(lambda kalimat: kalimat.replace("tgl. ", ""))
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].apply(lambda kalimat: kalimat.lstrip()[:2])
    bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].apply(lambda kalimat: ubah_format(kalimat))

    

    # bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].apply(lambda kalimat: kalimat.ljust(2, '0'))
    # bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].apply(lambda kalimat.string(): re.search(r'[0-9]{2}', kalimat))
    # bulan['tanggal_kejadian'] = bulan['tanggal_kejadian'].str.extract('(\d+)', expand=False)
    data_normaliasi_tanggal.append(bulan)


In [8]:
for bulan in data_normaliasi_tanggal:
    bulan.drop_duplicates()

In [9]:
# 6
# Membersihkan Data

for month in data_normaliasi_tanggal:
    length = len(month)
    last_kecamatan = ""
    last_kelurahan = ""

    i_container = []
    
    tanggal_container = []


    for i in range(length):
        if  i == 0:
            last_kecamatan = month.loc[i]["kecamatan"]
            last_kelurahan = month.loc[i]["kelurahan"]

            tanggal_container.append(month.loc[i]["tanggal_kejadian"])
            continue


        # Mendapatkan data terkini
        now_kecamatan = month.loc[i]["kecamatan"]
        now_kelurahan = month.loc[i]["kelurahan"]
        now_tanggal_kejadian = month.loc[i]["tanggal_kejadian"]

        # cek duplikat
        if now_kecamatan == last_kecamatan \
            and now_kelurahan == last_kelurahan \
            and now_tanggal_kejadian in tanggal_container:
                #Hapus baris ke i (variabel) di bulan 'month' (variabel)
                i_container.append(i)

        elif now_kecamatan == last_kecamatan \
            and now_kelurahan == last_kelurahan:
                tanggal_container.append(now_tanggal_kejadian)
    
        else:
            last_kecamatan = now_kecamatan
            last_kelurahan = now_kelurahan

            tanggal_container = []

    for _ in range(len(i_container)):
        i = i_container.pop()
        month.drop(i, inplace=True)


In [10]:
print(data_normaliasi_tanggal[10]['tanggal_kejadian'])

0      16
1      18
4      17
5      17
6      17
7      24
8      17
9      17
15     01
16     17
17     18
18     17
21     17
22     17
24     16
25     17
26     18
27     16
33     18
34     18
39     01
40     08
45     08
46     08
47     08
48     08
49     24
50     08
51     08
59     08
60     08
62     25
63     25
64     08
65     08
66     08
67     08
68     08
69     08
72     08
73     08
83     25
84     01
85     01
86     01
92     01
93     01
94     12
96     01
110    08
111    12
112    12
114    25
Name: tanggal_kejadian, dtype: object


In [11]:
# 7
# making a dictionary of kelurahan, where kecamatan : [kelurahan]
kecamatan = list()
kelurahan = dict()
for bulan in data_normaliasi_tanggal:
    kecamatan_bulan_n = bulan['kecamatan'].drop_duplicates()
    for i in kecamatan_bulan_n:
        if i not in kecamatan:
            kecamatan.append(i)
    num = int()
    for enum, kec in enumerate(kecamatan_bulan_n):
        kelurahan[kec] = list(bulan.loc[bulan['kecamatan'] == kec, 'kelurahan'].drop_duplicates())

In [12]:
print(kecamatan)

['JOHAR BARU', 'KEMAYORAN', 'SAWAH BESAR', 'TANAH ABANG', 'CILINCING', 'KELAPA GADING', 'KOJA', 'PENJARINGAN', 'PADEMANGAN', 'CENGKARENG', 'GROGOL PETAMBURAN', 'KALIDERES', 'KEBON JERUK', 'KEMBANGAN', 'PALMERAH', 'CILANDAK', 'JAGAKARSA', 'KEBAYORAN BARU', 'KEBAYORAN LAMA', 'MAMPANG PRAPATAN', 'PANCORAN', 'PASAR MINGGU', 'PESANGGRAHAN', 'SETIABUDI', 'TEBET', 'CAKUNG', 'CIPAYUNG', 'CIRACAS', 'DUREN SAWIT', 'JATINEGARA', 'KRAMAT JATI', 'MAKASAR', 'MATRAMAN', 'PASAR REBO', 'PULO GADUNG', 'SENEN', 'MENTENG', 'CEMPAKA PUTIH', 'GAMBIR', 'TANJUNG PRIOK', 'TAMAN SARI', 'TAMBORA', 'KEPULAUAN SERIBU SELATAN', 'KEPULAUAN SERIBU UTARA']


In [13]:
# 8
total = int()
for val in kelurahan.values():
    total += len(val)
print(total)

81


In [14]:
# for enum, kel in enumerate(kelurahan, 1):
#     print("bulan " + str(calendar.month_name[enum]))
#     for kec in kecamatan:
#         print("kecamatan : " + kec)
#         print("kelurahan : " + ", ".join(kelurahan[kec]))
#         print("_________")
        

In [15]:
# 9
# kasus banjir per-kelurahan
kasus_banjir_per_bulan = list()
banjir_per_kelurahan = dict()

for bulan in data_normaliasi_tanggal: 
    for kec in kecamatan:
        for kel in kelurahan[kec]:
            kejadian = len(bulan[bulan['kelurahan'] == kel])
            # print(kel + " " + str(kejadian))
            banjir_per_kelurahan[kel] = kejadian
    banjir_per_kelurahan = pd.Series(banjir_per_kelurahan).reset_index()
    banjir_per_kelurahan.columns = ['kelurahan', 'kejadian']
    kasus_banjir_per_bulan.append(banjir_per_kelurahan)
    banjir_per_kelurahan = dict()

In [16]:
print(data_normaliasi_tanggal[10])

    kota_administrasi         kecamatan       kelurahan tanggal_kejadian
0       Jakarta Utara       PENJARINGAN     KAPUK MUARA               16
1       Jakarta Utara       PENJARINGAN     KAPUK MUARA               18
4       Jakarta Utara       PENJARINGAN     KAPUK MUARA               17
5       Jakarta Utara       PENJARINGAN     PENJARINGAN               17
6       Jakarta Utara       PENJARINGAN     PENJARINGAN               17
7       Jakarta Utara       PENJARINGAN     PENJARINGAN               24
8       Jakarta Utara       PENJARINGAN           PLUIT               17
9       Jakarta Utara       PENJARINGAN           PLUIT               17
15      Jakarta Utara         CILINCING         ROROTAN               01
16      Jakarta Utara         CILINCING         MARUNDA               17
17      Jakarta Utara         CILINCING         MARUNDA               18
18      Jakarta Utara         CILINCING         MARUNDA               17
21      Jakarta Utara        PADEMANGAN           A

In [17]:
# 10
for setiap_bulan in kasus_banjir_per_bulan:
    print(setiap_bulan.nlargest(5, "kejadian"))

          kelurahan  kejadian
20            PLUIT         5
62  CIPINANG MELAYU         4
30      LEBAK BULUS         3
41         RAWAJATI         3
58   KAMPUNG MELAYU         3
         kelurahan  kejadian
22           KAPUK        10
25      TEGAL ALUR         9
59     BIDARA CINA         8
61          CAWANG         8
58  KAMPUNG MELAYU         6
         kelurahan  kejadian
58  KAMPUNG MELAYU        10
61          CAWANG         9
59     BIDARA CINA         8
42   PEJATEN TIMUR         6
34   PONDOK PINANG         5
          kelurahan  kejadian
34    PONDOK PINANG         3
42    PEJATEN TIMUR         3
46       KEBON BARU         3
58   KAMPUNG MELAYU         3
62  CIPINANG MELAYU         3
         kelurahan  kejadian
42   PEJATEN TIMUR         4
41        RAWAJATI         3
46      KEBON BARU         3
58  KAMPUNG MELAYU         3
34   PONDOK PINANG         2
      kelurahan  kejadian
20        PLUIT         7
13      MARUNDA         3
22        KAPUK         3
77   PULAU PAR